In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:

 (x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=10000)

print(f"Training samples: {len(x_train)}, Test samples: {len(x_test)}")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training samples: 25000, Test samples: 25000


In [3]:
# Ensure all reviews are the same length
maxlen = 200  # truncate/pad to 200 words

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [4]:
model = keras.Sequential([
    layers.Embedding(input_dim=10000, output_dim=32, input_length=maxlen),
    layers.SimpleRNN(64),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# Explicitly build the model (for summary to show parameters)
model.build(input_shape=(None, maxlen))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 32)        │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 326,273 (1.24 MB)

 Trainable params: 326,273 (1.24 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(x_train, y_train,
                    epochs=5,
                    batch_size=64,
                    validation_split=0.2)

Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 25s 73ms/step - accuracy: 0.5412 - loss: 0.6852 - val_accuracy: 0.6958 - val_loss: 0.5844
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 39s 68ms/step - accuracy: 0.7926 - loss: 0.4532 - val_accuracy: 0.6808 - val_loss: 0.5908
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 43s 74ms/step - accuracy: 0.8181 - loss: 0.4055 - val_accuracy: 0.7448 - val_loss: 0.5159
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 71ms/step - accuracy: 0.8890 - loss: 0.2745 - val_accuracy: 0.7652 - val_loss: 0.5127
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 41s 69ms/step - accuracy: 0.9300 - loss: 0.1877 - val_accuracy: 0.8212 - val_loss: 0.5073


In [7]:
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.8117 - loss: 0.5343
Test Accuracy: 0.8142


In [8]:
# Load the word index mapping used by Keras IMDb dataset
word_index = keras.datasets.imdb.get_word_index()

# Reserve first indices (as per Keras's convention)
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index[""] = 0
word_index[""] = 1
word_index[""] = 2
word_index[""] = 3

# Create reverse mapping (integers to words) if needed
reverse_word_index = {value: key for (key, value) in word_index.items()}




1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [9]:
def encode_review(text):
    tokens = text.lower().split()
    encoded = [1]  # Start token
    for word in tokens:
        index = word_index.get(word, 2)  # Use 2 for
        encoded.append(index)
    return keras.preprocessing.sequence.pad_sequences([encoded], maxlen=200)


In [10]:
def predict_sentiment(text):
    encoded = encode_review(text)
    prediction = model.predict(encoded)[0][0]
    label = "Positive " if prediction > 0.5 else "Negative "
    print(f"Review: {text}")
    print(f"Prediction: {prediction:.4f} — {label}")

In [11]:
predict_sentiment("The movie was really good.")
predict_sentiment("It was so boring and waste of money.")
predict_sentiment("This movie was a waste of time.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Review: The movie was really good.
Prediction: 0.0173 — Negative 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Review: It was so boring and waste of money.
Prediction: 0.0053 — Negative 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Review: This movie was a waste of time.
Prediction: 0.0044 — Negative 


In [12]:
model.save('sentiment_analysis_model.keras')

In [13]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [14]:
!wget -q -O - ipv4.icanhazip.com

34.16.183.161


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py
⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://little-dancers-clap.loca.lt


In [ ]:
y